Modeling main objectives:
1) maintain the huge size of the data set which will be challenging when appling diffrent modeling approch
2) appling three diffrent models and comparing there rusullts using two Evaluation Metrics
   - we are going to use model Accuracy scare range from 0 to 1 A higher score indicates a better fit. 
   - We are going to use Mean absolute error (MAE): MAE is a measure of the average absolute difference between the predicted values and the actual values.
 
 * Compare the Model Accuracy score to MAE scores for different models. The model with the lowest MAE and the highest model score we are going to select.
 * checking the emportancy of the features for the model.
3) Tune the algorithm selected the best model wiht highest evalutions from the above step using cross valdiation to select the best parameters for the model.

In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score,mean_absolute_error,precision_score,recall_score
from imblearn.under_sampling import RandomUnderSampler
import xgboost

In [2]:
# loading the dataset.
df = pd.read_csv('../data/processed/processed_droped_outliers_data.csv')
df

,age_group,length_of_stay,type_of_admission,ccs_diagnosis_description,ccs_procedure_description,apr_drg_description,apr_mdc_description,apr_severity_of_illness_description,apr_risk_of_mortality,gender_M,apr_medical_surgical_description_Surgical,emergency_department_indicator_Y
0,3,5,8.63,16.72,7.57,19.22,18.55,1,1,0,0,1
1,1,8,8.63,16.72,7.57,19.22,18.55,2,1,0,0,1
2,2,8,8.63,16.72,7.57,19.22,18.55,1,1,1,0,1
3,3,4,8.63,6.22,7.57,4.97,8.57,1,1,0,0,1
4,3,3,8.63,6.89,7.57,6.20,8.61,2,2,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
722448,3,3,8.63,9.32,6.79,5.76,7.81,3,3,0,1,1
722449,1,1,8.63,4.97,8.38,5.04,5.84,3,3,0,0,1
722450,4,3,8.63,7.68,4.88,7.40,7.81,3,3,1,0,1
722451,4,2,8.63,14.06,6.65,17.10,12.42,4,4,0,1,1


In [3]:
# Geitting Information about the Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722453 entries, 0 to 722452
Data columns (total 12 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   age_group                                  722453 non-null  int64  
 1   length_of_stay                             722453 non-null  int64  
 2   type_of_admission                          722453 non-null  float64
 3   ccs_diagnosis_description                  722453 non-null  float64
 4   ccs_procedure_description                  722453 non-null  float64
 5   apr_drg_description                        722453 non-null  float64
 6   apr_mdc_description                        722453 non-null  float64
 7   apr_severity_of_illness_description        722453 non-null  int64  
 8   apr_risk_of_mortality                      722453 non-null  int64  
 9   gender_M                                   722453 non-null  int64  
 10  apr_medi

In [4]:
# statistical description for numerical features
df.describe()

,age_group,length_of_stay,type_of_admission,ccs_diagnosis_description,ccs_procedure_description,apr_drg_description,apr_mdc_description,apr_severity_of_illness_description,apr_risk_of_mortality,gender_M,apr_medical_surgical_description_Surgical,emergency_department_indicator_Y
count,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000,722453.000000
mean,2.981853,8.571092,8.557930,8.563032,8.565470,8.565019,8.562137,2.438311,2.072825,0.486364,0.275615,0.658912
std,0.955242,11.142255,0.225746,3.713710,3.704387,5.175920,2.787466,0.915473,0.998609,0.499814,0.446824,0.474075
min,1.000000,1.000000,5.060000,2.000000,1.500000,1.880000,4.160000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,8.630000,6.480000,6.890000,5.650000,7.330000,2.000000,1.000000,0.000000,0.000000,0.000000
50%,3.000000,5.000000,8.630000,7.760000,7.570000,7.400000,7.710000,2.000000,2.000000,0.000000,0.000000,1.000000
75%,4.000000,10.000000,8.630000,9.470000,9.940000,9.490000,8.610000,3.000000,3.000000,1.000000,1.000000,1.000000
max,4.000000,120.000000,8.680000,26.070000,40.530000,46.140000,18.550000,4.000000,4.000000,1.000000,1.000000,1.000000


In [5]:
#checking for duplication in data 
df.duplicated().sum()

0

## 1) Maintain the huge size of the data set which will be challenging when appling diffrent modeling approch:
##### Although the size of the data after cleaning and preprocessing decreese greatly to 143,500 entries instance and 7 features but it is still challenging to deal with it for limted pressoer laptob so we are going take small sample to run the models more efficently while maintaing the same characteristics of the data by usin straitify method.

In [6]:
# divede the data fram to x = features & y = target
x = df.drop("length_of_stay", axis = 1)
y = df["length_of_stay"]

In [7]:
# taking sample of the data using stratify to maintain data characteristics
main_x,sample_x,main_y,sample_y = train_test_split(x,y,test_size=0.03,random_state=42,stratify=y)

In [8]:
len(sample_y)

21674

In [9]:
# now we will split sample data to train and test sets 
x_train,x_test,y_train,y_test = train_test_split(sample_x,sample_y,test_size=0.3,random_state=42)

In [10]:
# Using Stander Scaler for x_train, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [11]:
# trying three different models to compare between them
models = {"lr": LinearRegression(),
          "lasso": Lasso(),
          "knn": KNeighborsRegressor(),
          "rf": RandomForestRegressor(),
         "xgbr": xgboost.XGBRegressor()}

In [12]:
# running the three different models and check the result using for loop
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train_scaled,y_train)
    y_train_pred = model.predict(x_train_scaled)
    y_test_pred = model.predict(x_test_scaled)
    print("Train Acurracy Score", round(model.score(x_train_scaled,y_train),2),"Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
    print("Train MAE Score: ", round(mean_absolute_error(y_train_pred,y_train),2),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
    print("*"*60)

-----------  lr  -------------
Train Acurracy Score 0.28 Test Acurracy Score 0.28
Train MAE Score:  5.18 Test MAE Score:  5.43
************************************************************
-----------  lasso  -------------
Train Acurracy Score 0.25 Test Acurracy Score 0.25
Train MAE Score:  5.19 Test MAE Score:  5.44
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.45 Test Acurracy Score 0.22
Train MAE Score:  4.43 Test MAE Score:  5.66
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.85 Test Acurracy Score 0.23
Train MAE Score:  2.17 Test MAE Score:  5.67
************************************************************
-----------  xgbr  -------------
Train Acurracy Score 0.65 Test Acurracy Score 0.23
Train MAE Score:  3.76 Test MAE Score:  5.46
************************************************************


##### checking the emportancy of the features for the random forest model. by making data frame for features importancy

In [13]:
feat_imp = list(model.feature_importances_)
col_name = list(x_train.columns)
feat_imp_df = pd.DataFrame({"col_name":col_name,"feat_imp":feat_imp})

In [14]:
feat_imp_df[['col_name','feat_imp']].sort_values(by='feat_imp')

,col_name,feat_imp
0,age_group,0.040589
2,ccs_diagnosis_description,0.048181
1,type_of_admission,0.053766
8,gender_M,0.055371
3,ccs_procedure_description,0.059190
5,apr_mdc_description,0.059997
10,emergency_department_indicator_Y,0.061665
7,apr_risk_of_mortality,0.061797
9,apr_medical_surgical_description_Surgical,0.143003
4,apr_drg_description,0.196195


Dropping the apr_medical_surgical_description_Surgical as it is show least importancy and check the model for any improvement.

In [15]:
# dropping apr_medical_surgical_description_Surgical from both x_train & x_test
x_train.drop(['apr_medical_surgical_description_Surgical'],axis=1,inplace=True)
x_test.drop(['apr_medical_surgical_description_Surgical'],axis=1,inplace=True)

In [16]:
# Using Stander Scaler for x_train, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [17]:
# trying three different models to compare between them
models = {"lr": LinearRegression(),
          "knn": KNeighborsRegressor(),
          "rf": RandomForestRegressor()}

In [18]:
# running the three different models and check the result using for loop
for name,model in models.items():
    print("----------- ", name, " -------------")
    model.fit(x_train_scaled,y_train)
    y_train_pred = model.predict(x_train_scaled)
    y_test_pred = model.predict(x_test_scaled)
    print("Train Acurracy Score", model.score(x_train_scaled,y_train))
    print("Train MAE Score: ", mean_absolute_error(y_train_pred,y_train))
    print('-' * 20)
    print("Test Acurracy Score", model.score(x_test_scaled,y_test))
    print("Test MAE Score: ", mean_absolute_error(y_test_pred,y_test))
    print("*"*60)

-----------  lr  -------------
Train Acurracy Score 0.274052194464368
Train MAE Score:  5.174737812312937
--------------------
Test Acurracy Score 0.27808152153251786
Test MAE Score:  5.421234868054954
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.44288600789095156
Train MAE Score:  4.467444466416189
--------------------
Test Acurracy Score 0.21361142276973255
Test MAE Score:  5.69912348147009
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.8545094477157085
Train MAE Score:  2.1844713095150112
--------------------
Test Acurracy Score 0.2288062917767627
Test MAE Score:  5.674455501778622
************************************************************


* The models show No significant imporvement imporvement 

#### We are going to use polynomial Features with diffrent degree and check if there are any improvement

In [19]:
for i in [2,3,4]:
    print("----------- ", f" Poly degree No.{i}", " -------------")
    poly2 = PolynomialFeatures(degree=i)
    x_train_poly = poly2.fit_transform(x_train)
    x_test_poly = poly2.transform(x_test)
    for name,model in models.items():
        print("----------- ", name, " -------------")
        model.fit(x_train_poly,y_train)
        y_train_pred = model.predict(x_train_poly)
        y_test_pred = model.predict(x_test_poly)
        print("Train Acurracy Score", round(model.score(x_train_poly,y_train),2), "Test Acurracy Score", round(model.score(x_test_poly,y_test),2))
        print("Train MAE Score: ", round(mean_absolute_error(y_train_pred,y_train),2),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
        print("*"*60)

-----------   Poly degree No.2  -------------
-----------  lr  -------------
Train Acurracy Score 0.3 Test Acurracy Score 0.3
Train MAE Score:  5.0 Test MAE Score:  5.29
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.46 Test Acurracy Score 0.21
Train MAE Score:  4.42 Test MAE Score:  5.7
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.85 Test Acurracy Score 0.24
Train MAE Score:  2.18 Test MAE Score:  5.67
************************************************************
-----------   Poly degree No.3  -------------
-----------  lr  -------------
Train Acurracy Score 0.33 Test Acurracy Score 0.3
Train MAE Score:  4.94 Test MAE Score:  5.33
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.46 Test Acurracy Score 0.2
Train MAE Score:  4.42 Test MAE Score:  5.73
*********************************

### Using Princeple Component Analasis after polynomial with different componant numper and check the results

In [20]:
# using n_componests 2,5,8,10 and checking the results.
pca=PCA(n_components=5)
x_train_pca10 =pca.fit_transform(x_train)
x_test_new_pca10 =pca.transform(x_test)

In [21]:
for i in [2,3,4]:
    print("----------- ", i, " -------------")
    poly2 = PolynomialFeatures(degree=i)
    x_train_poly = poly2.fit_transform(x_train)
    x_test_poly = poly2.transform(x_test)
    
    for n in [2,5,8,10]:
        print("----------- ", n, " -------------")
        pca=PCA(n_components=2)
        x_train_pca =pca.fit_transform(x_train)
        x_test_pca =pca.transform(x_test)
        
        for name,model in models.items():
            print("----------- ", name, " -------------")
            model.fit(x_train_pca,y_train)
            y_train_pred = model.predict(x_train_pca)
            y_test_pred = model.predict(x_test_pca)
            print("Train Acurracy Score", round(model.score(x_train_pca,y_train),2), "Test Acurracy Score", round(model.score(x_test_pca,y_test),2))
            print("Train MAE Score: ", round(mean_absolute_error(y_train_pred,y_train),2),"Test MAE Score: ", round(mean_absolute_error(y_test_pred,y_test),2))
            print("*"*60)

-----------  2  -------------
-----------  2  -------------
-----------  lr  -------------
Train Acurracy Score 0.22 Test Acurracy Score 0.23
Train MAE Score:  5.4 Test MAE Score:  5.65
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.41 Test Acurracy Score 0.16
Train MAE Score:  4.68 Test MAE Score:  6.02
************************************************************
-----------  rf  -------------
Train Acurracy Score 0.84 Test Acurracy Score 0.15
Train MAE Score:  2.33 Test MAE Score:  6.11
************************************************************
-----------  5  -------------
-----------  lr  -------------
Train Acurracy Score 0.22 Test Acurracy Score 0.23
Train MAE Score:  5.4 Test MAE Score:  5.65
************************************************************
-----------  knn  -------------
Train Acurracy Score 0.42 Test Acurracy Score 0.16
Train MAE Score:  4.68 Test MAE Score:  6.02
*******************************

In [22]:
# First we need to change the target to numerical to string using dictionery and map it to y train & test
str_target = {1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',10:'10',11:'11',12:'12',13:'13',14:'14',15:'15',16:'16',17:'17',18:'18',19:'19',20:'20'}
y_train = y_train.map(str_target)
y_test = y_test.map(str_target)

In [23]:
y_train.value_counts().sort_values()

20     115
19     120
18     149
17     166
16     201
15     227
14     276
13     315
12     348
11     394
10     470
9      565
8      721
7      911
6     1054
5     1249
4     1503
1     1637
3     1711
2     1812
Name: length_of_stay, dtype: int64

In [24]:
# trying three different classification models to compare between them
models = {"lg": LogisticRegression(),
          "knnc": KNeighborsClassifier(),
          "rfc": RandomForestClassifier()}

In [25]:
for name,model in models.items():
        print("----------- ", name, " -------------")
        model.fit(x_train_scaled,y_train)
        y_train_pred = model.predict(x_train_scaled)
        y_test_pred = model.predict(x_test_scaled)
        print("Train Acurracy Score", round(model.score(x_train_scaled,y_train),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
        print("Train Precision Score: ", round(precision_score(y_train,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
        print("Train Recall Score: ", round(recall_score(y_train,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
        print("*"*60)

-----------  lg  -------------


ValueError: Input contains NaN

In [ ]:
# using random under sampler to maintain inbalance of the target class
resampler = RandomUnderSampler(sampling_strategy='auto')

In [ ]:
x_train_resample,y_train_resample = resampler.fit_resample(x_train,y_train)

In [ ]:
y_train_resample.value_counts().sort_values()

In [ ]:
# Using Stander Scaler for x_train_resampled, x_test
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_resample)
x_test_scaled = scaler.transform(x_test)

In [ ]:
for name,model in models.items():
        print("----------- ", name, " -------------")
        model.fit(x_train_scaled,y_train_resample)
        y_train_pred = model.predict(x_train_resample)
        y_test_pred = model.predict(x_test_scaled)
        print("Train Acurracy Score", round(model.score(x_train_scaled,y_train_resample),2), "Test Acurracy Score", round(model.score(x_test_scaled,y_test),2))
        print("Train Precision Score: ", round(precision_score(y_train_resample,y_train_pred,average='micro'),2),"Test Precision Score: ", round(precision_score(y_test,y_test_pred,average='micro'),2))
        print("Train Recall Score: ", round(recall_score(y_train_resample,y_train_pred,average='micro'),2),"Test Recall Score: ", round(recall_score(y_test,y_test_pred,average='micro'),2))
        print("*"*60)

### The Models evaluations results above indicate that there is both under fitting and over fitting in the same time which mean we have to go back again to the preprocessing steps and try to deal with that for example by dropping outliers and also try different way of Categorical incoding like binary encoding.